In [1]:
import numpy as np

In [2]:
data = np.loadtxt('../data/channel_Re590.txt',skiprows=8)


In [5]:
data.shape[0]

59

In [11]:
tke = data[:, 0]
eps = data[:, 1]
grad_u_flat = data[:, 2:11]
stresses_flat = data[:, 11:]

# Reshape grad_u and stresses to num_points X 3 X 3 arrays
num_points = data.shape[0]
grad_u = np.zeros((num_points, 3, 3))
stresses = np.zeros((num_points, 3, 3))
for i in range(3):
    for j in range(3):
        grad_u[:, i, j] = grad_u_flat[:, i*3+j]
        stresses[:, i, j] = stresses_flat[:, i*3+j]


In [12]:
grad_u[0]

array([[ 2.0821e-04,  4.8370e+02, -9.2901e-01],
       [-3.5325e-07, -3.2383e-04,  2.3919e-05],
       [ 4.6207e-08,  1.5474e-04,  1.4803e-06]])

In [17]:
cap = 7
eps = np.maximum(eps, 1e-8)
tke_eps = tke / eps

Sij = np.zeros((num_points, 3, 3))
Rij = np.zeros((num_points, 3, 3))

In [21]:

for i in range(num_points):
    Sij[i, :, :] = 0.5 * (grad_u[i, :, :] + np.transpose(grad_u[i, :, :]))
    Rij[i, :, :] = 0.5 * (grad_u[i, :, :] - np.transpose(grad_u[i, :, :]))


In [23]:
Sij[0]

array([[ 2.08210000e-04,  2.41850000e+02, -4.64504977e-01],
       [ 2.41850000e+02, -3.23830000e-04,  8.93295000e-05],
       [-4.64504977e-01,  8.93295000e-05,  1.48030000e-06]])

In [24]:
Rij[0]

array([[ 0.00000000e+00,  2.41850000e+02, -4.64505023e-01],
       [-2.41850000e+02,  0.00000000e+00, -6.54105000e-05],
       [ 4.64505023e-01,  6.54105000e-05,  0.00000000e+00]])

In [ ]:

# Sij[Sij > cap] = cap
# Sij[Sij < -cap] = -cap
# Rij[Rij > cap] = cap
# Rij[Rij < -cap] = -cap

# Because we enforced limits on maximum Sij values, we need to re-enforce trace of 0
for i in range(num_points):
    Sij[i, :, :] = Sij[i, :, :] - 1./3. * np.eye(3)*np.trace(Sij[i, :, :])
# return Sij, Rij